In [345]:
import datetime
import os
import re
import numpy as np
from seabird import fCNV
import seapy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.colors as colors
import scipy.io as sio
import cmaps
from matplotlib.colors import LinearSegmentedColormap
from cotede import datasets, qctests

In [666]:
class insitu_workspace():
    def __init__(self,MY_ROOT,worksapce_name, workspace_detail=None):
        self.my_root = MY_ROOT
        self.workspace_name = worksapce_name
        self.workspace_detail = workspace_detail
        self.workspace_subdir = os.path.join(self.my_root, self.workspace_name)
        print('Workspace: ' + self.workspace_subdir)
        self.ctd_files = seapy.list_files(os.path.join(self.workspace_subdir+'/CTD/*acfld.cnv'))
        self.ctd_stationinfo = os.path.join(self.workspace_subdir,self.workspace_name+'-station.txt')
        self.isempty = True
        
        
    class Data():
        def __init__(self):
            self.filetype = 'ctd data structure'
            




    def pull_downcast(self,ctd_file,VAR):
        def lanczos_filter(data,Cf,Nf,M):
            coef = sp.signal.firwin(M+1, Cf/Nf,width = 2/len(data),window='lanczos', pass_zero='lowpass')
            return sp.signal.filtfilt(coef,1.0,data)

        dt = 0.04 #sec
        fs = 1/(dt)
        Nf = fs/2
        Cf = fs/1500
        M = 100
        print(ctd_file)
        profile = fCNV(ctd_file)
        time    = profile['timeS'][:]
        if VAR == 'PSAL2':
            var = profile.data[-6][:]
        else:
            var     = profile[VAR][:]
        pres    = profile['PRES'][:]
        depth   = profile['DEPTH'][:]
        # smooth the noise further
        pres_conv = np.convolve(pres,np.ones(24*6)/24/6,mode='full')[72:-71]
        pres_conv = seapy.filt.bandpass(pres_conv,dt,low_cutoff=dt*200,order=8)
        diff_pres = np.diff(pres_conv)
    #    diff_pres_conv = np.convolve(diff_pres,np.ones(24*6)/24/6,mode='full')[72:-71]
        diff_pres_conv = diff_pres.copy()
        count_down = 0
        count_up = 0
        for i in range(1,len(diff_pres_conv)):
            # print(diff_pres[i])
            # print(diff_pres[i]*diff_pres[i-1])
            if (diff_pres_conv[i]*diff_pres_conv[i-1]) < 0 :
                if diff_pres_conv[i-1]>0:
                    if  not count_down:
                        start_down = i
                    else:
                        start_down = np.r_[start_down, i]
                    count_down+=1
                else:
                    if not count_up :
                        start_up = i
                    else:
                        start_up = np.r_[start_up, i]
                    count_up+=1


    #        print(start_up)
    #        print(start_down)


        # print(pres[start_down[1]])
        larger = 1
        smaller = -1
        equal = 0
    #    print(len(start_down))
    #    print(len(start_up))
        if len(start_up) > len(start_down):
            end_indx = len(start_down)
            status = larger
        elif len(start_up) < len(start_down):
            end_indx = len(start_up)
            status = smaller
        else:
            status = equal


        if start_down[0] < start_up[0]:
            if status == equal:
                mono_duration = start_up[:-1] - start_down[1:]
            elif status == larger:
                mono_duration = start_up[:end_indx-1] - start_down[1:]
            elif status == smaller:
                mono_duration = start_up[:] - start_down[1:]
    #        print(mono_duration)
            downcast_index = np.argmax(np.abs(mono_duration))
    #        print(downcast_index)
            if ((pres[start_down[downcast_index+1]] -
                      pres[start_up[downcast_index]]) < 5):
                mono_duration[downcast_index] = 0
                downcast_index = np.argmax(np.abs(mono_duration))
            mono_duration[downcast_index] = 0

            downcast2_index = np.argmax(np.abs(mono_duration))

            if (downcast2_index > downcast_index)\
                and ((pres[start_down[downcast2_index+1]] - 
                      pres[start_up[downcast2_index]]) > 5):
                time_downcast = np.r_[
                                      time[start_up[downcast_index]:start_down[downcast_index+1]],
                                      time[start_up[downcast2_index]:start_down[downcast2_index+1]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast_index]:start_down[downcast_index+1]],
                                      pres[start_up[downcast2_index]:start_down[downcast2_index+1]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast_index]:start_down[downcast_index+1]],
                                      depth[start_up[downcast2_index]:start_down[downcast2_index+1]]
                                      ]
                var_downcast = np.r_[
                                       var[start_up[downcast_index]:start_down[downcast_index+1]],
                                       var[start_up[downcast2_index]:start_down[downcast2_index+1]]
                                    ]
            elif (downcast2_index < downcast_index)\
                and ((((pres[start_down[downcast2_index+1]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres) *0.4) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index+1]]) <np.max(time)*0.1))\
                or (((pres[start_down[downcast2_index+1]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres) *0.2) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index+1]]) <np.max(time)*0.05))\
                or((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index+1]]) <np.max(time)*0.005)): # judge that the interval between two downcast is small enough
                time_downcast = np.r_[
                                      time[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                      time[start_up[downcast_index]:start_down[downcast_index+1]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                      pres[start_up[downcast_index]:start_down[downcast_index+1]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                      depth[start_up[downcast_index]:start_down[downcast_index+1]]
                                      ]
                var_downcast = np.r_[
                                       var[start_up[downcast2_index]:start_down[downcast2_index+1]],
                                       var[start_up[downcast_index]:start_down[downcast_index+1]]
                                    ]
            else:

                time_downcast = time[start_up[downcast_index]:start_down[downcast_index+1]]
                pres_downcast = pres[start_up[downcast_index]:start_down[downcast_index+1]]
                depth_downcast = depth[start_up[downcast_index]:start_down[downcast_index+1]]
                var_downcast = var[start_up[downcast_index]:start_down[downcast_index+1]]
                
            



        else:
            if status == equal:
                mono_duration = start_up - start_down
            elif status == larger:
                mono_duration = start_up[:end_indx] - start_down
            elif status == smaller:
                mono_duration = start_up[:] - start_down[:end_indx]

    #        print(mono_duration)
            downcast_index = np.argmax(np.abs(mono_duration))
    #        print(downcast_index)
            if ((pres[start_down[downcast_index]] -
                      pres[start_up[downcast_index]]) < 5):
                mono_duration[downcast_index] = 0
                downcast_index = np.argmax(np.abs(mono_duration))
            mono_duration[downcast_index] = 0
            downcast2_index = np.argmax(np.abs(mono_duration))
            if (downcast2_index > downcast_index)\
                and ((pres[start_down[downcast2_index]] - 
                      pres[start_up[downcast2_index]]) > 5):
                time_downcast = np.r_[
                                      time[start_up[downcast_index]:start_down[downcast_index]],
                                      time[start_up[downcast2_index]:start_down[downcast2_index]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast_index]:start_down[downcast_index]],
                                      pres[start_up[downcast2_index]:start_down[downcast2_index]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast_index]:start_down[downcast_index]],
                                      depth[start_up[downcast2_index]:start_down[downcast2_index]]
                                      ]
                
                var_downcast = np.r_[
                                       var[start_up[downcast_index]:start_down[downcast_index]],
                                       var[start_up[downcast2_index]:start_down[downcast2_index]]
                                    ]
            elif (downcast2_index < downcast_index)\
                 and ((((pres[start_down[downcast2_index]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres)*0.4) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index]]) <np.max(time)*0.1))\
              or (((pres[start_down[downcast2_index]] - 
                  pres[start_up[downcast2_index]]) > np.max(pres)*0.2) \
                and ((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index]]) <np.max(time)*0.05))\
                or((time[start_up[downcast_index]] -
                  time[start_down[downcast2_index]]) <np.max(time)*0.005)): # judge that the interval between two downcast is small enough
                time_downcast = np.r_[
                                      time[start_up[downcast2_index]:start_down[downcast2_index]],
                                      time[start_up[downcast_index]:start_down[downcast_index]]

                                      ]
                pres_downcast = np.r_[
                                      pres[start_up[downcast2_index]:start_down[downcast2_index]],
                                      pres[start_up[downcast_index]:start_down[downcast_index]]
                                      ]
                depth_downcast = np.r_[
                                      depth[start_up[downcast2_index]:start_down[downcast2_index]],
                                      depth[start_up[downcast_index]:start_down[downcast_index]]
                                      ]
                var_downcast = np.r_[
                                       var[start_up[downcast2_index]:start_down[downcast2_index]],
                                       var[start_up[downcast_index]:start_down[downcast_index]]
                                    ]
            else:

                time_downcast = time[start_up[downcast_index]:start_down[downcast_index]]
                pres_downcast = pres[start_up[downcast_index]:start_down[downcast_index]]
                depth_downcast = depth[start_up[downcast_index]:start_down[downcast_index]]
                var_downcast = var[start_up[downcast_index]:start_down[downcast_index]]
            
            
        return time_downcast.data, pres_downcast.data, depth_downcast.data, var_downcast.data

    def pull_ctdinfos(self):
        self.ctd_infos = {}
        ctdid = open(self.ctd_stationinfo)
        info = ctdid.readline().split()
        cid = 0
        while info:
            self.ctd_infos[info[1]]={}
            self.ctd_infos[info[1]]['TIME'] = (datetime.datetime.strptime(info[2],'%Y%m%d%H%M') - 
                                              datetime.datetime(1970,1,1)).total_seconds()/3600/24
            self.ctd_infos[info[1]]['LONGITUDE']=info[3]
            self.ctd_infos[info[1]]['LATITUDE']=info[4]
            self.ctd_infos[info[1]]['DEPTH'] = info[5]
            self.ctd_infos[info[1]]['CTDID'] = cid
            cid +=1
            info = ctdid.readline().split()
        ctdid.close()
        
    def pull_alldata(self,VARS, interp2meter=0,refine_withqc=0):
        if self.isempty:
            self.data = {}
            self.isempty = False
        for iv, var in enumerate(VARS):
            self.data[var] = self.Data()
            for ic, ctd_file in enumerate(self.ctd_files):
                # print(ctd_file)
                ctdid = re.findall(r"%s/CTD/(.*)acfld.cnv"%(self.workspace_subdir),ctd_file)[0]
                print(ctdid)
                if ic == 0:
                    _, self.data[var].pres, self.data[var].depth, self.data[var].value = self.pull_downcast(ctd_file,var)
                    #remove negative depth
                    valid_depth = np.where(self.data[var].depth>0)
                    self.data[var].pres = self.data[var].pres[valid_depth]
                    self.data[var].depth = self.data[var].depth[valid_depth]
                    self.data[var].value = self.data[var].value[valid_depth]
                    self.data[var].station = np.tile(ctdid,[len(self.data[var].value),]).ravel()
                    self.data[var].time = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(self.data[var].value),]).ravel()
                    self.data[var].lon = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(self.data[var].value),]).ravel()
                    self.data[var].lat = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(self.data[var].value),]).ravel()
                    if interp2meter:
                        print('interp is activate')
                        
                        data_len = int(np.floor(0.95*len(self.data[var].value)))
                        pres_cut = self.data[var].pres[:data_len] 
                        # splitted based on pressure
                        # cut the tail of data
                        pres_levels = np.arange(np.ceil(np.min(pres_cut)),
                                                   np.floor(np.max(pres_cut)),
                                                   )
                        
                        self.data[var].station_i = np.tile(ctdid,[len(pres_levels),]).ravel()
                        self.data[var].time_i = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(pres_levels),]).ravel()
                        self.data[var].lon_i = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(pres_levels),]).ravel()
                        self.data[var].lat_i = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(pres_levels),]).ravel()
                        self.data[var].depth_i = seapy.seawater.depth(pres_levels, lat=float(self.ctd_infos[ctdid]['LATITUDE']))
                        self.data[var].pres_i = pres_levels.copy()
                        self.data[var].value_i = np.interp(pres_levels,pres_cut,self.data[var].value[:data_len])
                    if refine_withqc:
                        
                        
                else:
                    _, thispres, thisdepth, thisvalue = self.pull_downcast(ctd_file,var)
                    #remove negative depth
                    valid_depth = np.where(thisdepth>0)
                    thispres = thispres[valid_depth]
                    thisvalue = thisvalue[valid_depth]
                    thisdepth = thisdepth[valid_depth]
                    thisstation = np.tile(ctdid,[len(thisvalue),]).ravel()
                    thistime = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(thisvalue),]).ravel()
                    thislon = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(thisvalue),]).ravel()
                    thislat = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(thisvalue),]).ravel()
                    self.data[var].station  = np.r_[self.data[var].station ,thisstation]
                    self.data[var].time  = np.r_[self.data[var].time ,thistime]
                    self.data[var].pres  = np.r_[self.data[var].pres ,thispres]
                    self.data[var].depth = np.r_[self.data[var].depth,thisdepth]
                    self.data[var].value = np.r_[self.data[var].value,thisvalue]
                    self.data[var].lon   = np.r_[self.data[var].lon,thislon]
                    self.data[var].lat   = np.r_[self.data[var].lat,thislat]
                    if interp2meter:
                        print('interp is activate')
                        
                        data_len = int(np.floor(0.95*len(thisvalue)))
                        pres_cut = thispres[:data_len] 
                        # splitted based on pressure
                        # cut the tail of data
                        pres_levels = np.arange(np.ceil(np.min(pres_cut)),
                                                   np.floor(np.max(pres_cut)),
                                                   )
                        
                        thisstation_i = np.tile(ctdid,[len(pres_levels),]).ravel()
                        thistime_i = np.tile(float(self.ctd_infos[ctdid]['TIME']),[len(pres_levels),]).ravel()
                        thislon_i = np.tile(float(self.ctd_infos[ctdid]['LONGITUDE']),[len(pres_levels),]).ravel()
                        thislat_i = np.tile(float(self.ctd_infos[ctdid]['LATITUDE']),[len(pres_levels),]).ravel()
                        thisdepth_i = seapy.seawater.depth(pres_levels, lat=float(self.ctd_infos[ctdid]['LATITUDE']))
                        thispres_i = pres_levels.copy()
                        thisvalue_i = np.interp(pres_levels,pres_cut,thisvalue[:data_len])
                        self.data[var].station_i = np.r_[self.data[var].station_i, thisstation_i]
                        self.data[var].time_i = np.r_[self.data[var].time_i, thistime_i]
                        self.data[var].lon_i = np.r_[self.data[var].lon_i, thislon_i]
                        self.data[var].lat_i = np.r_[self.data[var].lat_i, thislat_i]
                        self.data[var].depth_i = np.r_[self.data[var].depth_i, thisdepth_i]
                        self.data[var].pres_i = np.r_[self.data[var].pres_i, thispres_i]
                        self.data[var].value_i = np.r_[self.data[var].value_i,thisvalue_i]
                    
    
        
            
                    
                    
                   
                
                
            
        

In [667]:
test =insitu_workspace('/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS','IOCAS2012SPRING','IOCAS2012SPRING')

Workspace: /Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING


In [668]:
test.pull_ctdinfos()

In [669]:
# must pull_ctdinfos before pull_alldata
test.pull_alldata(VARS=['TEMP','PSAL2'],interp2meter=True)

3300-01
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3300-01acfld.cnv
interp is activate
3300-02
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3300-02acfld.cnv
interp is activate
3300-03
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3300-03acfld.cnv
interp is activate
3300-04
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3300-04acfld.cnv
interp is activate
3300-05
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3300-05acfld.cnv
interp is activate
3300-06
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3300-06acfld.cnv
interp is activate
3400-01
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3400-01acfld.cnv
interp is activate
3400-02
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/IOCAS2012SPRING/CTD/3400-02acfld.cnv
interp is activate
3400-03
/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSIT

In [738]:
np.unique(test.data['TEMP'].station[station_position])

array(['3400-04'], dtype='<U7')

In [737]:
plt.plot(test.data['TEMP'].lat[station_position])

In [674]:
test.data['TEMP'].value

array([25.6706, 25.6736, 25.675 , ..., 18.1849, 18.1849, 18.1849])

In [792]:
#temp profile
%matplotlib
import matplotlib.tri as tri
# VAR='PSAL2'
VAR='TEMP'
# -------------NSFC2012SPRING-YS
# stations = ['E01','E02','E03','E04','E05']
# stations = ['H01','H02','H03','H04','H05','H06','H07','H08']
# stations = ['H38','H39','H40','H41','H42']
# stations = ['H33','H34','H35','H36']
# stations = ['H25','H26','H27','H28','H29']
# stations = ['H10','H11','H12','H13','H14','H15','H16','H17','H18','H19']
# stations = ['HF1','HF2','HF3','H43','H44']
# -------------NSFC2012SPRING-ES
# stations = ['DH3-01','RB-10','RB-16','DH3-02','DH3-03','DH3-04','DH3-05','DH3-06','DH3-07','DH3-08',]
# stations = ['DH4-01','DH4-02','DH4-03','DH4-04','DH4-05','DH4-06','ZA-05','ZA-03','ZA-01']
# stations = ['GC-05','ZB-07','ZB-09','ZB-11','DH5-01','GC-06','ZB-12','DH5-02','DH5-03','DH5-04','DH5-05','DH5-06']
# stations = ['DH6-05','DH6-04','DH6-03','DH6-02','ZC-17','DH6-01','ZC-15','ZC-13']
# stations = ['ZD-19','ZD-21','DH7-01','ZD-24','DH7-02','DH7-03','DH7-04','DH7-05']
# stations = ['DH8-05','DH8-04','DH8-03','DH8-02','ZE-27','DH8-01','ZE-25']
# stations = ['GC-07', 'DH9-01','DH9-02','DH9-03','DH9-04','DH9-05','DH9-06']
# -------------IOCAS2012SPRING
# stations = ['DH9-1','DH9-2','DH9-3','DH9-4','DH9-5','DH9-6']
# stations = ['DH8-5','DH8-4','DH8-3','DH8-2','DH8-1']
# stations = ['DH7-1','DH7-2','DH7-3','DH7-4','DH7-5']
# stations = ['DH6-5','DH6-4','DH6-3','DH6-2','DH6-1']
# stations = ['DH5-1','DH5-2','DH5-3','DH5-4','DH5-5','DH5-6']
# stations = ['DH4-6','DH4-5','DH4-4','DH4-3','DH4-2','DH4-1']
# stations = ['DH3-1','DH3-2','DH3-3','DH3-4','DH3-5','DH3-6','DH3-7','DH3-8']
# stations = ['DH2-7','DH2-6','DH2-5','DH2-4','DH2-3','DH2-2','DH2-1']
# stations = ['CJ-1','CJ-2','CJ-3','CJ-4','CJ-5','CJ-6']
# stations = ['DH1-7','DH1-8']
# stations=['3300-06','3300-05','3300-04','3300-03','3300-02','3300-01']
# stations = ['3400-01','3400-02','3400-03','3400-04','3400-05','3400-06','3400-07','3400-08']
# stations = ['3500-10','3500-09','3500-08','3500-07','3500-06','3500-05','3500-04','3500-03','3500-02','3500-01']
# stations = ['3600-01','3600-02','3600-03','3600-04','3600-05','3600-06','3600-07','3600-08']
# stations =['B-08','B-07','B-06','B-05','B-04','B-02','B-01']
stations = ['3875-01','3875-02','3875-03','3875-04','3875-05']
fig, ax = plt.subplots()
i = 0
for istation in stations:
    # print(istation)
    # print(test.ctd_infos[istation]['CTDID'])
    current_position = np.where(test.data[VAR].station == istation)[0]
    # print(current_position)
    if i == 0:
        station_position = current_position.copy()
        
    else:
        station_position = np.r_[station_position,current_position]
        
    i+=1
    
lon_station = test.data[VAR].lon[station_position]
lat_station = test.data[VAR].lat[station_position]
depth_station = test.data[VAR].depth[station_position]
data_station = test.data[VAR].value[station_position]
cntr = ax.tricontourf(lon_station,depth_station,data_station,levels=30,cmap='jet')

ax.scatter(np.unique(lon_station),np.zeros_like(np.unique(lon_station)), color='k')
ax.invert_yaxis()

fig.colorbar(cntr, ax=ax)

Using matplotlib backend: MacOSX


In [791]:
fig, ax = plt.subplots()
i = 0
for istation in stations:
    # print(istation)
    # print(test.ctd_infos[istation]['CTDID'])
    current_position = np.where(test.data[VAR].station_i == istation)[0]
    # print(current_position)
    if i == 0:
        station_position = current_position.copy()
        
    else:
        station_position = np.r_[station_position,current_position]
        
    i+=1
    
lon_station = test.data[VAR].lon_i[station_position]
lat_station = test.data[VAR].lat_i[station_position]
depth_station = test.data[VAR].depth_i[station_position]
data_station = test.data[VAR].value_i[station_position]
cntr = ax.tricontourf(lon_station,depth_station,data_station,levels=30,cmap='jet')

ax.scatter(np.unique(lon_station),np.zeros_like(np.unique(lon_station)),color='k')
ax.invert_yaxis()
fig.colorbar(cntr, ax=ax)

In [699]:
depth_station

array([ 0.99338132,  1.98675782,  2.9801295 ,  3.97349636,  4.9668584 ,
        5.96021562,  6.95356802,  7.94691561,  8.94025837,  9.93359632,
       10.92692944, 11.92025775, 12.91358124, 13.90689992, 14.90021377,
       15.89352281, 16.88682703, 17.88012643, 18.87342102, 19.86671078,
       20.85999573, 21.85327587, 22.84655118, 23.83982168, 24.83308737,
       25.82634823, 26.81960428, 27.81285552, 28.80610194, 29.79934354,
       30.79258033, 31.7858123 , 32.77903946, 33.7722618 , 34.76547932,
       35.75869203, 36.75189993, 37.74510301, 38.73830128, 39.73149473,
       40.72468337, 41.71786719, 42.7110462 , 43.7042204 , 44.69738978,
       45.69055435, 46.68371411, 47.67686905, 48.67001918, 49.6631645 ,
       50.656305  , 51.64944069, 52.64257157, 53.63569763, 54.62881889,
       55.62193533, 56.61504696, 57.60815377, 58.60125578, 59.59435297,
       60.58744536, 61.58053293, 62.57361569, 63.56669364, 64.55976678,
       65.5528351 , 66.54589862, 67.53895733, 68.53201122, 69.52

In [786]:
station = 'B-03'
fig, ax = plt.subplots()
current_position = np.where(test.data[VAR].station == station)[0]
    # print(current_position)

station_position = current_position.copy()


    
lon_station = test.data[VAR].lon[station_position]
lat_station = test.data[VAR].lat[station_position]
pres_station = test.data[VAR].pres[station_position]
depth_station = test.data[VAR].depth[station_position]
data_station = test.data[VAR].value[station_position]
ax.plot(data_station,depth_station)
ax.invert_yaxis()

In [764]:
station = '3600-06'
fig, ax = plt.subplots()
current_position = np.where(test.data[VAR].station_i == station)[0]
    # print(current_position)

station_position = current_position.copy()


    
lon_station = test.data[VAR].lon_i[station_position]
lat_station = test.data[VAR].lat_i[station_position]
depth_station = test.data[VAR].depth_i[station_position]
data_station = test.data[VAR].value_i[station_position]
ax.plot(data_station,depth_station)
ax.invert_yaxis()

In [87]:
position = np.where(test.data['TEMP'].time==test.data['TEMP'].time[1])

In [91]:
seapy.seawater.depth(test.data['TEMP'].pres[position],test.data['TEMP'].lat[position])

array([ 1.6733532 ,  1.65547769,  1.64058143, ..., 14.99712619,
       14.98719599, 14.9772658 ])

In [92]:
test.data['TEMP'].depth[position]

array([ 1.673,  1.655,  1.641, ..., 14.997, 14.987, 14.977])

In [88]:
position

(array([   0,    1,    2, ..., 1231, 1232, 1233]),)

In [679]:
test.data['TEMP'].station

array(['3300-01', '3300-01', '3300-01', ..., 'DH9-6', 'DH9-6', 'DH9-6'],
      dtype='<U7')

In [54]:
test.ctd_files

['/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E01acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E02acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E03acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E04acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/E05acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H01acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H02acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H03acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H04acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING-YS/CTD/H05acfld.cnv',
 '/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/INSITU_OBS/NSFC2012SPRING

In [67]:
sst_obs = seapy.roms.obs.obs(filename='/Volumes/TO_1/roms4dvar_ecs/i4dvar_outputs/NOAA_GHRSST/geopolar_sst_201205.nc')

In [61]:
# merge sst and in-situ togethor
grdfile = '/Volumes/TO_1/outputs_SCORRECTION/ocean_ecs_fwd_15461_outer1.nc'
dt = 1/24
# reftime = datetime.datetime(1970,1,1)


In [13]:
grid = seapy.model.asgrid(grdfile)

In [63]:
data = [seapy.roms.obs.raw_data("TEMP", "CTD", test.data['TEMP'].value,None,0.1)]
        
insitu_obs = seapy.roms.obs.gridder(grid,test.data['TEMP'].time,
                                    test.data['TEMP'].lon,test.data['TEMP'].lat,
                                    -test.data['TEMP'].depth, data,dt,title="cruise obs")

Output()

In [68]:
sst_obs.add(insitu_obs)

In [69]:
sst_obs.to_netcdf('./combined_sst.nc')

In [66]:
pwd

'/Volumes/TO_1/roms4dvar_ecs/i4dvar_analysis/obs_processing'

array([371095.81666667, 371095.81666667, 371095.81666667, ...,
       371246.7       , 371246.7       , 371246.7       ])